In [ ]:
from IPython.core.debugger import set_trace

from keras.utils.vis_utils import model_to_dot
from IPython.display import SVG
import pydot_ng as pydot


import sys
sys.path.append('../financial_utils/')

import keras
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

%matplotlib notebook

import matplotlib
import matplotlib.pyplot as plt

import performance as per

# DATASET

### Load and Compute Returns

In [ ]:
price_table = pd.read_csv('history_files/SPY.csv')
vol_table = pd.read_csv('history_files/VIX.csv')

# Build Tick Table
tick_table = price_table[["Date","AdjClose"]]
tick_table.columns = ["Date", "Tick"]
# Get Return Table
returns = per.tick2ret(tick_table)
returns_table = pd.DataFrame({'Date':tick_table.Date[1:], 'Return': returns[:,0]})
returns_table = returns_table.set_index('Date')
# Drop columns of Vol Table
vol_table = vol_table[["Date", "Close"]]
vol_table.columns = ["Date", "Vol"]
vol_table.Vol = vol_table.Vol/100
vol_table = vol_table.set_index('Date')
# InnerJoin
retvol_table = pd.concat([returns_table,vol_table], join='inner', axis=1)


In [ ]:
T_x = 14
n_fields = retvol_table.shape[1]
T_y = 5
T_stride = 1

### Generate Dataset

In [ ]:
def gen_dataset(T_x, T_y, T_stride, retvol_table):
    """
    
    
    return
        x E (m, T_x)
        y E (m, T_y)
    """
    
    m = int(np.floor((returns.shape[0] - T_x) / T_stride))
    
    x = np.nan * np.ones((m, T_x, retvol_table.shape[1]))
    y = np.nan * np.ones((m, T_y, retvol_table.shape[1]))
    
    for i in range(m):
        x[i,:,0] = retvol_table.Return[i*T_stride:i*T_stride+T_x].transpose()
        x[i,:,1] = retvol_table.Vol[i*T_stride:i*T_stride+T_x].transpose()
        y[i,:,0] = retvol_table.Return[i*T_stride:i*T_stride+T_y].transpose()
        y[i,:,1] = retvol_table.Vol[i*T_stride:i*T_stride+T_y].transpose()

    return x, y, m

In [ ]:
def normalize_data(x, params=None):
    """
    
    return 
        x_norm
        x_norm_param = (mean, std)
    """
    if params == None:
        mean = np.mean(x, axis=(0,1)) 
        std = np.std(x, axis=(0,1))
    else:
        mean = params[0]
        std = params[1]
    x_norm_param = (mean, std)
    x_norm = (x - mean)/std
    
    return x_norm, x_norm_param

In [ ]:
def denormalize_data(x_norm, x_norm_param):
    """"""
    
    mean = x_norm_param[0]
    std = x_norm_param[1]
    x = x_norm * std + mean
    
    return x

In [ ]:
x, y, m = gen_dataset(T_x,T_y,T_stride,retvol_table)
x, x_norm_param = normalize_data(x)
y, _ = normalize_data(y, x_norm_param)
print('Training Example: '+str(m))
print('X Shape: '+str(x.shape))
print('Y Shape: '+str(y.shape))

### Train / Val / Test

In [ ]:
train_end = int(m * 0.7)

val_end = train_end + int(0.15 * m)

x_train = x[0:train_end]
x_val = x[train_end:val_end]
x_test = x[val_end:]

y_train = y[0:train_end]
y_val = y[train_end:val_end]
y_test = y[val_end:]

# Keras Model

In [ ]:
keras.backend.clear_session()

n_a = 100

encoder_LSTM = keras.layers.LSTM(units = n_a, return_state=True)
decoder_LSTM = keras.layers.LSTM(units = n_a, return_state=True, return_sequences=True)

flatter = keras.layers.Flatten()
dense = keras.layers.Dense(units = 100, activation='relu')
relu_out = keras.layers.Dense(units = 2, activation='relu')
concatenator = keras.layers.Lambda(lambda x: keras.backend.stack(x, axis=1))

In [ ]:
def define_model(T_x, T_y, n_fields, n_a):
    
    x_input = keras.layers.Input(shape=(T_x, n_fields))
    output_seq = list()
    
    decoder_outputs, encoder_h, encoder_c = decoder_LSTM(x_input)
    decoder_outputs_flat = flatter(decoder_outputs)
    out = dense(decoder_outputs_flat)
    out = relu_out(out)
    print(out.shape)
    #decoder_input = keras.layers.Input(shape=(1,n_a))
    deco_input = last_slicor(out)
    decoder_h = encoder_h
    decoder_c = encoder_c
    
    for _ in range(T_y):
        decoder_outputs, decoder_h, decoder_c = decoder_LSTM(deco_input, initial_state=[decoder_h, decoder_c])
        decoder_outputs_flat = flatter(decoder_outputs)
        out = dense(decoder_outputs_flat)
        out = relu_out(out)
        
        output_seq.append(out)
        deco_input = decoder_outputs
        
    output_seq = concatenator(output_seq)
    model = keras.models.Model(inputs=[x_input, decoder_input], outputs=output_seq)
    return model

In [ ]:
model = define_model(T_x, T_y, n_fields, n_a)
model.summary()

In [ ]:
optim = keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=optim, loss='mse', metrics=['accuracy'])
decoder_in = np.zeros((x_train.shape[0],1,n_a))
model.fit(x=[x_train,decoder_in], y=y_train, shuffle=True, epochs = 10)

In [ ]:
decoder_in = np.zeros((x_test.shape[0],1,n_a))
model.evaluate(x=[x_test,decoder_in], y=y_test)

In [ ]:
indexes = np.random.randint(0,x_test.shape[0], size=1)
x_in = x_test[indexes,:,:]
decoder_in = np.zeros((x_in.shape[0],1,n_a))
y_true = y_test[indexes,:,:]
y_pred = denormalize_data(model.predict(x=[x_in,decoder_in]), x_norm_param)
y_pred

In [ ]:
x_norm_param

## SNIPPETS

### Norm - Denorm Check

In [ ]:
a, b , m = gen_dataset(T_x,T_y,T_stride,retvol_table)
x, x_norm_param = normalize_data(a)
y, _ = normalize_data(b, x_norm_param)
x = denormalize_data(x, x_norm_param)
y = denormalize_data(y, x_norm_param)

plt.figure()
plt.hist((b-y).flatten(), 100)